# MSI - zadanie 1

Membership functions class

In [1]:
import numpy as np

In [ ]:
class TriangularMembershipFunction():
    def __init__(self, left, peak, right):
        if left < peak and peak < right:
            self.left = left
            self.peak = peak
            self.right = right
        else:
            raise Exception("Left point must be less than peak and peak must be less than right point!")
        self.variable_name = None
        
    def calculate(self, x):
        if x <= self.left or x > self.right:
            return 0
        elif x > self.left and x <= self.peak:
            return (x-self.left)/(self.peak-self.left)
        elif self.peak < x < self.right:
            return (self.right - x) / (self.right - self.peak)
        elif x == self.right:
            return 1
        
class TrapezoidalMembershipFunction():
    def __init__(self, left_point, left_upper, right_upper, right_point):
        if left_point > left_upper or left_upper > right_upper or right_upper > right_point:
            raise Exception("Wrong parameters for the function!")
        
        self.left_point = left_point
        self.left_upper = left_upper
        self.right_upper = right_upper
        self.rigth_point = right_point
        
    def calculate(self, x):
        if x <= self.left_point or x >= self.rigth_point:
            return 0
        elif self.left_point <= x and x <= self.left_upper:
            return (x-self.left_point)/(self.left_upper-self.left_point)
        elif self.left_upper < x and x <= self.right_upper:
            return 1
        else:
            return (self.rigth_point-x)/(self.rigth_point-self.right_upper)

Antecendent class

In [13]:
class Antecedent():
    def __init__(self, membership_funcs, name, range):
        self.name = name
        self.range = range
        for degree in membership_funcs:
            membership_funcs[degree].variable_name = name
        self.membership_funcs = membership_funcs

In [14]:
quality = Antecedent({"poor" : TriangularMembershipFunction(0, 0.0001, 5), "average" : TriangularMembershipFunction(0, 5, 10), "good" : TriangularMembershipFunction(5, 9.9999, 10)}, "quality", np.linspace(0, 10, 100))
service = Antecedent({"poor" : TriangularMembershipFunction(0, 0.0001, 5), "average" : TriangularMembershipFunction(0, 5, 10), "good" : TriangularMembershipFunction(5, 9.9999, 10)}, "service", np.linspace(0, 10, 100))

Rule class

In [15]:
class orRule():
    def __init__(self, antecedent, value_function):
        self.antecedent = antecedent
        self.value_function = value_function
        
    def calculate_weight(self, dict_of_inputs):
        values = []
        for func in self.antecedent:
            input_value = dict_of_inputs[func.variable_name]
            values.append(func.calculate(input_value))
        return max(values)
    
class andRule():
    def __init__(self, antecedent, value_function):
        self.antecedent = antecedent
        self.value_function = value_function
        
    def calculate_weight(self, dict_of_inputs):
        values = []
        for func in self.antecedent:
            input_value = dict_of_inputs[func.variable_name]
            values.append(func.calculate(input_value))
            
        return min(values)

In [23]:
rule1 = orRule([quality.membership_funcs["poor"], service.membership_funcs["poor"]], 0)
rule2 = andRule([service.membership_funcs["average"]], 13)
rule3 = orRule([quality.membership_funcs["good"], service.membership_funcs["good"]], 15)

list_of_rules = [rule1, rule2, rule3]

In [24]:
class SugenoInferenceSystem():
    def __init__(self, list_of_rules):
        self.list_of_rules = list_of_rules
        self.inputs = {}
        
    def calculate_rule_strengths(self):
        weights = {}
        for rule in self.list_of_rules:
            w = rule.calculate_weight(self.inputs)
            weights[w] = rule.value_function
        self.weights = weights
        return weights

    def defuzzify_output(self):
        output_top = 0
        output_down = 0
        for weight in self.weights:
            output_top = output_top + weight*self.weights[weight]
            output_down = output_down + weight
        if output_down == 0:
            return 0
        return output_top/output_down
    
    def calculate(self):
        self.calculate_rule_strengths()
        return self.defuzzify_output()
    

In [25]:
system = SugenoInferenceSystem(list_of_rules)

system.inputs["quality"] = 2
system.inputs["service"] = 5

system.calculate()

13.0

In [26]:
system = SugenoInferenceSystem(list_of_rules)

system.inputs["quality"] = 0
system.inputs["service"] = 0

system.calculate()

0

In [27]:
system = SugenoInferenceSystem(list_of_rules)

system.inputs["quality"] = 10
system.inputs["service"] = 10

system.calculate()

15.0